In [36]:
# 分析に使うImport群
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [37]:
train = pd.read_csv('train.csv')

test = pd.read_csv('test.csv')
sample = pd.read_csv('sample.csv', header=None)


In [38]:
# 課題が終わった状態
train['year'] = train['datetime'].apply(lambda x : x.split('-')[0])
train['month'] = train['datetime'].apply(lambda x : x.split('-')[1])

test['year'] = test['datetime'].apply(lambda x : x.split('-')[0])
test['month'] = test['datetime'].apply(lambda x : x.split('-')[1])

train['fun'] = train['remarks'].apply(lambda x: 1 if x == 'お楽しみメニュー' else 0)
test['fun'] = test['remarks'].apply(lambda x: 1 if x == 'お楽しみメニュー' else 0)

trainX = train[['year', 'month', 'fun', 'temperature']]
testX = test[['year', 'month', 'fun', 'temperature']]

Y = train['y']


In [39]:
model_linear = LinearRegression()
model_linear.fit(trainX, Y)
pred = model_linear.predict(trainX)


In [40]:
train['pred'] = pred

In [41]:
train['res'] = train['y'] - train['pred']

In [42]:
train.sort_values(by='res')

,datetime,y,week,soldout,name,kcal,remarks,event,payday,weather,precipitation,temperature,year,month,fun,pred,res
27,2013-12-26,80,木,0,酢豚,NaN,NaN,NaN,NaN,曇,--,7.3,2013,12,0,139.603689,-59.603689
103,2014-4-25,80,金,0,中華丼,NaN,お楽しみメニュー,NaN,NaN,晴れ,--,20.8,2014,4,1,124.562163,-44.562163
82,2014-3-27,55,木,1,ロース甘味噌焼き,315.0,NaN,NaN,NaN,曇,1.5,12.4,2014,3,0,94.672542,-39.672542
0,2013-11-18,90,月,0,厚切りイカフライ,NaN,NaN,NaN,NaN,快晴,--,19.8,2013,11,0,128.125605,-38.125605
145,2014-6-27,74,金,0,牛丼,NaN,お楽しみメニュー,NaN,NaN,雨,0,25.4,2014,6,1,110.381030,-36.381030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2014-3-17,129,月,0,鶏の唐揚げおろしソース,382.0,NaN,NaN,NaN,快晴,--,14.0,2014,3,0,92.665120,36.334880
55,2014-2-17,137,月,1,チンジャオロース,416.0,NaN,NaN,NaN,晴れ,--,11.6,2014,2,0,99.881151,37.118849
85,2014-4-1,125,火,1,メンチカツ,377.0,NaN,NaN,NaN,薄曇,--,16.1,2014,4,0,85.825482,39.174518
13,2013-12-5,171,木,0,鶏のカッシュナッツ炒め,NaN,NaN,NaN,NaN,快晴,--,13.8,2013,12,0,131.448539,39.551461


In [43]:
# 標準偏差
train['res'].std()

17.33389004831175

In [44]:
# ここからが追加
# weatherから快晴のみを判断

In [82]:
train['clear sky'] = train['weather'].apply(lambda x: 1 if x in['快晴'] else 0)
test['clear sky'] = test['weather'].apply(lambda x: 1 if x in['快晴'] else 0)

train['weather int'] = train['weather'].map({'快晴': 0, '晴れ': 1, '薄曇': 2, '曇': 3, '雷電': 4, '雪': 5, '雨': 6})
test['weather int'] = test['weather'].map({'快晴': 0, '晴れ': 1, '薄曇': 2, '曇': 3, '雷電': 4, '雪': 5, '雨': 6})


In [46]:
trainX = train[['year', 'month', 'fun', 'temperature', 'clear sky']]
testX = test[['year', 'month', 'fun', 'temperature', 'clear sky']]


In [47]:
def predict_for_submit(filename):
    model_linear.fit(trainX, Y)
    predict = model_linear.predict(trainX)
    train['pred'] = predict
    train['res'] = train['y'] - train['pred']
    print('std: {0}'.format(train['res'].std()))
    
    predict = model_linear.predict(testX)
    sample[1] = predict
    sample.to_csv(filename, index=None, header=None)

In [48]:
predict_for_submit('submit6.csv')

std: 17.308782512329977


In [49]:
## 曜日を追加

In [50]:
weather_categories = set(train['weather'].unique().tolist() + test['weather'].unique().tolist())

train['weather'] = pd.Categorical(train['weather'], categories=weather_categories)
test['weather'] = pd.Categorical(test['weather'], categories=weather_categories)


In [51]:
dummy = pd.get_dummies(train['weather'], prefix='', prefix_sep='')
train = pd.concat([train, dummy], axis=1)

dummy = pd.get_dummies(test['weather'], prefix='', prefix_sep='')
test = pd.concat([test, dummy], axis=1)


In [52]:
train.head()

,datetime,y,week,soldout,name,kcal,remarks,event,payday,weather,...,res,clear sky,weather int,雨,雷電,薄曇,快晴,曇,雪,晴れ
0,2013-11-18,90,月,0,厚切りイカフライ,NaN,NaN,NaN,NaN,快晴,...,-37.367689,1,0.0,0,0,0,1,0,0,0
1,2013-11-19,101,火,1,手作りヒレカツ,NaN,NaN,NaN,NaN,快晴,...,-29.828690,1,0.0,0,0,0,1,0,0,0
2,2013-11-20,118,水,0,白身魚唐揚げ野菜あん,NaN,NaN,NaN,NaN,快晴,...,-14.682798,1,0.0,0,0,0,1,0,0,0
3,2013-11-21,120,木,1,若鶏ピリ辛焼,NaN,NaN,NaN,NaN,快晴,...,-13.053620,1,0.0,0,0,0,1,0,0,0
4,2013-11-22,130,金,1,ビッグメンチカツ,NaN,NaN,NaN,NaN,快晴,...,-1.941155,1,0.0,0,0,0,1,0,0,0


In [53]:
trainX = train[['year', 'month', 'fun', 'temperature', '薄曇', '快晴', '雪', '雷電', '曇', '晴れ', 'clear sky']]
testX = test[['year', 'month', 'fun', 'temperature', '薄曇', '快晴', '雪', '雷電', '曇', '晴れ', 'clear sky']]


In [54]:
# 提出用データを作成する

In [55]:
predict_for_submit('submit7.csv')

std: 17.10567439173806


In [56]:
# 曜日を数値化。天気も数値化

In [122]:
print(train['weather'].value_counts())
print(train['week'].value_counts())

曇     53
快晴    53
晴れ    50
薄曇    25
雨     24
雪      1
雷電     1
Name: weather, dtype: int64
木    43
水    43
火    41
金    41
月    39
Name: week, dtype: int64


In [60]:
train['week int'] = train['week'].map({'月': 0, '火': 1, '水': 2, '木': 3, '金': 4})
test['week int'] = test['week'].map({'月': 0, '火': 1, '水': 2, '木': 3, '金': 4})


In [67]:
trainX = train[['year', 'month', 'fun', 'temperature','clear sky', 'week int']]
testX = test[['year', 'month', 'fun', 'temperature', 'clear sky', 'week int']]
# 
predict_for_submit('submit8.csv')

std: 15.626314127800248


In [123]:
# score下がった
trainX = train[['year', 'month', 'fun', 'temperature','clear sky', 'week int', 'weather int']]
testX = test[['year', 'month', 'fun', 'temperature', 'clear sky', 'week int', 'weather int']]

predict_for_submit('submit9.csv')

std: 15.646812241470162


In [116]:
def event_check(event):
    if event == 'キャリアアップ支援セミナー':
        return 1
    elif event == 'ママの会':
        return 2
    else:
        return 0

In [121]:
test['event'].value_counts()

キャリアアップ支援セミナー    2
ママの会             1
Name: event, dtype: int64

In [118]:
# イベントの影響の有無を確認
train['event flg'] = train['event'].apply(event_check)
test['event flg'] = test['event'].apply(event_check)
train

,datetime,y,week,soldout,name,kcal,remarks,event,payday,weather,...,weather int,雨,雷電,薄曇,快晴,曇,雪,晴れ,week int,event flg
0,2013-11-18,90,月,0,厚切りイカフライ,NaN,NaN,NaN,NaN,快晴,...,0,0,0,0,1,0,0,0,0,0
1,2013-11-19,101,火,1,手作りヒレカツ,NaN,NaN,NaN,NaN,快晴,...,0,0,0,0,1,0,0,0,1,0
2,2013-11-20,118,水,0,白身魚唐揚げ野菜あん,NaN,NaN,NaN,NaN,快晴,...,0,0,0,0,1,0,0,0,2,0
3,2013-11-21,120,木,1,若鶏ピリ辛焼,NaN,NaN,NaN,NaN,快晴,...,0,0,0,0,1,0,0,0,3,0
4,2013-11-22,130,金,1,ビッグメンチカツ,NaN,NaN,NaN,NaN,快晴,...,0,0,0,0,1,0,0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,2014-9-24,59,水,1,白身魚のマスタード焼き,408.0,NaN,NaN,NaN,曇,...,3,0,0,0,0,1,0,0,2,0
203,2014-9-25,50,木,0,牛カルビ焼き肉,394.0,NaN,NaN,NaN,曇,...,3,0,0,0,0,1,0,0,3,0
204,2014-9-26,45,金,0,ランチビュッフェ,NaN,スペシャルメニュー（800円）,キャリアアップ支援セミナー,NaN,晴れ,...,1,0,0,0,0,0,0,1,4,1
205,2014-9-29,56,月,1,豚肉と玉子の炒め,404.0,NaN,NaN,NaN,快晴,...,0,0,0,0,1,0,0,0,0,0


In [119]:
# 
trainX = train[['year', 'month', 'fun', 'temperature','clear sky', 'week int', 'event flg']]
testX = test[['year', 'month', 'fun', 'temperature', 'clear sky', 'week int', 'event flg']]

predict_for_submit('submit10.csv')

std: 15.625616947013812
